In [2]:
from gensim.scripts.glove2word2vec import glove2word2vec

# file = '/home/stirunag/pre-trained_word_embeddings/glove/glove.6B.50d.txt'
# glove2word2vec(glove_input_file=file, word2vec_output_file="gensim_glove.6B.50d.txt")

###Finally, read the word2vec txt to a gensim model using KeyedVectors:

from gensim.models.keyedvectors import KeyedVectors

# glove_model = KeyedVectors.load_word2vec_format("gensim_glove.6B.50d.txt", binary=False)


glove_model = KeyedVectors.load_word2vec_format("/home/stirunag/pre-trained_word_embeddings/GoogleNews-vectors-negative300.bin", binary=True)

/home/stirunag/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
import numpy as np
from scipy import spatial

index2word_set = set(glove_model.wv.index2word)


def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


s1_afv = avg_feature_vector('this is a sentence', model=glove_model,   num_features=300, index2word_set=index2word_set)
s2_afv = avg_feature_vector('this is also sentence', model=glove_model,num_features=300, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)


s1_afv = avg_feature_vector('apple cider banana', model=glove_model,   num_features=300, index2word_set=index2word_set)
s2_afv = avg_feature_vector('this is also sentence', model=glove_model,num_features=300, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

0.9596675038337708
0.1212424486875534


/home/stirunag/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [54]:
from skimage import feature
from PIL import Image
import numpy as np

def kullback_leibler_divergence(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    filt = np.logical_and(p != 0, q != 0)
    return np.sum(p[filt] * np.log2(p[filt] / q[filt]))

def concatenate_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec =  np.array([])  #np.zeros((num_features, len(words)), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.append(feature_vec, np.transpose(glove_model[word]))#np.add(feature_vec, model[word])
    
    return feature_vec.reshape((n_words, num_features))


def extract_1dlbp_cpu(input, neighborhood, p):
    """
    Extract the 1d lbp pattern
    """
    res = np.zeros(1 << (2 * neighborhood))
    for i in range(neighborhood, len(input) - neighborhood):
        left = input[i - neighborhood : i]
        right = input[i + 1 : i + neighborhood + 1]
        both = np.r_[left, right]
        res[np.sum(p [both >= input[i]])] += 1
    return res


def LBP_1D_avg_feature_vector(sentence, model, num_features, index2word_set, neighborhood, powers):
    words = sentence.split()
    feature_vec = np.zeros((1 << (2 * neighborhood), ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            LBP_hist = extract_1dlbp_cpu(model[word], neighborhood, powers)
            feature_vec = np.add(feature_vec, LBP_hist)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
    # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
 
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="var")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist
    
    
def chiSquared(p,q):
    return 0.5*np.sum((p-q)**2/(p+q+1e-6))    


In [41]:
s1_cfv = concatenate_feature_vector('apple banana mango doctor', glove_model,300, index2word_set)
s2_cfv = concatenate_feature_vector('this is sentence structure', glove_model,300, index2word_set)

desc = LocalBinaryPatterns(24, 8)                       

hist1 = desc.describe(Image.fromarray(s1_cfv))
hist2 = desc.describe(Image.fromarray(s2_cfv))

sim = 1 - chiSquared(hist1, hist2)
sim

1.0

In [18]:
kullback_leibler_divergence(hist1, hist2)

0.0

In [11]:
# # import the necessary packages
# from scipy.spatial import distance as dist
# import matplotlib.pyplot as plt
# import numpy as np
# import argparse
# import glob
# # import cv2

# # https://www.pyimagesearch.com/2014/07/14/3-ways-compare-histograms-using-opencv-python/

In [87]:
# 1D LBP experiment

neighborhood = 13
powers_ = 1 << np.array(range(0, 2 * neighborhood), dtype='int32')

s1_1D_cfv = LBP_1D_avg_feature_vector('this is sentence structure', glove_model,300, index2word_set,neighborhood,powers_)
s2_1D_cfv = LBP_1D_avg_feature_vector('this is sentence structure',glove_model,300, index2word_set,neighborhood,powers_)

sim = 1 - chiSquared(s1_1D_cfv, s2_1D_cfv)
print(sim)

print(kullback_leibler_divergence(s1_1D_cfv, s2_1D_cfv))

print(1 - spatial.distance.cosine(s1_1D_cfv, s2_1D_cfv))


1.0
0.0
1.0


In [47]:
sentence = 'apple banana mango doctor'
words = sentence.split()
feature_vec = np.zeros((2^8, ), dtype='float32')

for word in words:
    if word in index2word_set:
        LBP_hist = extract_1dlbp_cpu(glove_model[word], neighborhood, p)
        feature_vec = np.add(feature_vec, LBP_hist)
            
            

ValueError: operands could not be broadcast together with shapes (10,) (64,) 

In [49]:
p.shape

(6,)

In [30]:
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            LBP_hist = extract_1dlbp_cpu(model[word], neighborhood, powers)
            feature_vec = np.add(feature_vec, LBP_hist)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

NameError: name 'i' is not defined